## Get started!

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

### First I need to process the data a little bit.

In [3]:
stock = pd.read_csv('sphist.csv')
stock_raw = stock
print(stock.head())

         Date         Open         High          Low        Close  \
0  2015-12-07  2090.419922  2090.419922  2066.780029  2077.070068   
1  2015-12-04  2051.239990  2093.840088  2051.239990  2091.689941   
2  2015-12-03  2080.709961  2085.000000  2042.349976  2049.620117   
3  2015-12-02  2101.709961  2104.270020  2077.110107  2079.510010   
4  2015-12-01  2082.929932  2103.370117  2082.929932  2102.629883   

         Volume    Adj Close  
0  4.043820e+09  2077.070068  
1  4.214910e+09  2091.689941  
2  4.306490e+09  2049.620117  
3  3.950640e+09  2079.510010  
4  3.712120e+09  2102.629883  


In [4]:
for col in stock.columns:
    print(col, stock[col].dtype)

Date object
Open float64
High float64
Low float64
Close float64
Volume float64
Adj Close float64


In [5]:
stock['Date'] = pd.to_datetime(stock['Date'], format='%Y-%m-%d')
print(stock['Date'].dtype)

datetime64[ns]


In [6]:
stock = stock.sort_values(by='Date')
print(stock.head())

            Date   Open   High    Low  Close     Volume  Adj Close
16589 1950-01-03  16.66  16.66  16.66  16.66  1260000.0      16.66
16588 1950-01-04  16.85  16.85  16.85  16.85  1890000.0      16.85
16587 1950-01-05  16.93  16.93  16.93  16.93  2550000.0      16.93
16586 1950-01-06  16.98  16.98  16.98  16.98  2010000.0      16.98
16585 1950-01-09  17.08  17.08  17.08  17.08  2520000.0      17.08


Here I'm goint to calculate the mean of closing price for the past 5 days, 30 days and 365 days, and shift them down so it won't appear to be predicting price using today's price.

In [7]:
stock['close_5'] = stock['Close'].rolling(window=5).mean()
stock['close_30'] = stock['Close'].rolling(window=30).mean()
stock['close_365'] = stock['Close'].rolling(window=365).mean()
print(stock.head())

            Date   Open   High    Low  Close     Volume  Adj Close  close_5  \
16589 1950-01-03  16.66  16.66  16.66  16.66  1260000.0      16.66      NaN   
16588 1950-01-04  16.85  16.85  16.85  16.85  1890000.0      16.85      NaN   
16587 1950-01-05  16.93  16.93  16.93  16.93  2550000.0      16.93      NaN   
16586 1950-01-06  16.98  16.98  16.98  16.98  2010000.0      16.98      NaN   
16585 1950-01-09  17.08  17.08  17.08  17.08  2520000.0      17.08     16.9   

       close_30  close_365  
16589       NaN        NaN  
16588       NaN        NaN  
16587       NaN        NaN  
16586       NaN        NaN  
16585       NaN        NaN  


In [8]:
stock[['close_5', 'close_30', 'close_365']] = stock[['close_5', 'close_30', 'close_365']].shift(1)
stock = stock.dropna(axis=0)
print(stock.head(6))

            Date       Open       High        Low      Close     Volume  \
16224 1951-06-19  22.020000  22.020000  22.020000  22.020000  1100000.0   
16223 1951-06-20  21.910000  21.910000  21.910000  21.910000  1120000.0   
16222 1951-06-21  21.780001  21.780001  21.780001  21.780001  1100000.0   
16221 1951-06-22  21.549999  21.549999  21.549999  21.549999  1340000.0   
16220 1951-06-25  21.290001  21.290001  21.290001  21.290001  2440000.0   
16219 1951-06-26  21.299999  21.299999  21.299999  21.299999  1260000.0   

       Adj Close  close_5   close_30  close_365  
16224  22.020000   21.800  21.703333  19.447726  
16223  21.910000   21.900  21.683000  19.462411  
16222  21.780001   21.972  21.659667  19.476274  
16221  21.549999   21.960  21.631000  19.489562  
16220  21.290001   21.862  21.599000  19.502082  
16219  21.299999   21.710  21.564333  19.513617  


In [9]:
train = stock[stock['Date'] < datetime(year=2013, month=1, day=1)]
test = stock[stock['Date'] >= datetime(year=2013, month=1, day=1)]
print(train.tail(1), '\n\n\n')
print(test.head(1))

          Date         Open        High          Low        Close  \
739 2012-12-31  1402.430054  1426.73999  1398.109985  1426.189941   

           Volume    Adj Close      close_5     close_30    close_365  
739  3.204330e+09  1426.189941  1419.434009  1411.830001  1327.261562   



          Date         Open         High          Low        Close  \
738 2013-01-02  1426.189941  1462.430054  1426.189941  1462.420044   

           Volume    Adj Close      close_5     close_30    close_365  
738  4.202600e+09  1462.420044  1418.641992  1414.258667  1327.534055  


In [10]:
lr_1 = LinearRegression()
lr_3 = LinearRegression()

feature_1 = ['close_5']
feature_3 = ['close_5', 'close_30', 'close_365']
target = 'Close'

lr_1.fit(train[feature_1], train[target])
prediction_1 = lr_1.predict(test[feature_1])
mae_1 = mean_absolute_error(test[target], prediction_1)

lr_3.fit(train[feature_3], train[target])
prediction_3 = lr_3.predict(test[feature_3])
mae_3 = mean_absolute_error(test[target], prediction_3)

print('Mean Absolute Error for using only past 5 days mean closing price:\t', mae_1)
print('Mean Absolute Error for using 3 mean closing price:\t\t\t', mae_3)

Mean Absolute Error for using only past 5 days mean closing price:	 16.2678787545
Mean Absolute Error for using 3 mean closing price:			 16.1424396436


### The result is not too bad, but still have a lot of room to improve.

next I'm considering to use these ideas to improve my model.  
  
The average volume over the past five days.  
The average volume over the past year.  
The ratio between the average volume for the past five days, and the average volume for the past year.  
The standard deviation of the average volume over the past five days.  
The standard deviation of the average volume over the past year.  
The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.  
The year component of the date.  
The ratio between the lowest price in the past year and the current price.  
The ratio between the highest price in the past year and the current price.  
The year component of the date.  
The month component of the date.  
The day of week.  
The day component of the date.  
The number of holidays in the prior month.

In [95]:
############ Processing Cell ############

stock1 = stock_raw
stock1['Date'] = pd.to_datetime(stock1['Date'], format='%Y-%m-%d')
stock1 = stock1.sort_values(by='Date')

stock1['close_5'] = stock1['Close'].rolling(window=5).mean()
stock1['close_30'] = stock1['Close'].rolling(window=30).mean()
stock1['close_365'] = stock1['Close'].rolling(window=365).mean()
#########(got mae=16.1424)#########

stock1['volume_5'] = stock1['Volume'].rolling(window=5).mean()
stock1['volume_30'] = stock1['Volume'].rolling(window=30).mean()
stock1['volume_365'] = stock1['Volume'].rolling(window=365).mean()
#########(got mae=16.1213)#########

stock1['std_v5'] = stock1['Volume'].rolling(window=5).std()
stock1['std_v30'] = stock1['Volume'].rolling(window=30).std()
stock1['std_v365'] = stock1['Volume'].rolling(window=365).std()
#########(got mae=16.1393)#########

stock1['amplitude'] = stock1['High'] - stock1['Low']
stock1['amp_5'] = stock1['amplitude'].rolling(window=5).mean()
stock1['amp_30'] = stock1['amplitude'].rolling(window=30).mean()
stock1['amp_365'] = stock1['amplitude'].rolling(window=365).mean()
#########(got mae=16.0139)#########

stock1['weekdays'] = stock1['Date'].dt.dayofweek
stock1['months'] = stock1['Date'].dt.month
weekday_dummies = pd.get_dummies(stock1['weekdays'], prefix='weekdays')
month_dummies = pd.get_dummies(stock1['months'], prefix='months')
stock1 = pd.concat([stock1, weekday_dummies, month_dummies], axis=1)
day_list = list(weekday_dummies.columns.values) + list(month_dummies.columns.values)
#########(got mae=15.9851)#########

feature_cols = ['close_5', 'close_30', 'close_365',
               'volume_5', 'volume_30', 'volume_365',
               'std_v5', 'std_v30', 'std_v365',
               'amp_5', 'amp_30', 'amp_365']

stock1[feature_cols] = stock1[feature_cols].shift(1)

stock1 = stock1.dropna(axis=0)

train1 = stock1[stock['Date'] < datetime(year=2013, month=1, day=1)]
test1 = stock1[stock['Date'] >= datetime(year=2013, month=1, day=1)]
print(stock1.shape)
print(day_list)

(16225, 39)
['weekdays_0', 'weekdays_1', 'weekdays_2', 'weekdays_3', 'weekdays_4', 'months_1', 'months_2', 'months_3', 'months_4', 'months_5', 'months_6', 'months_7', 'months_8', 'months_9', 'months_10', 'months_11', 'months_12']


In [107]:
############ Adjusting Cell ############

lr1 = LinearRegression()

feature1 = ['close_5', 'close_30', 'close_365',
            'volume_5', 'volume_30', 'volume_365',
            'std_v5', 'std_v30', 'std_v365',
            'amp_5', 'amp_30', 'amp_365'] + day_list
target = 'Close'

lr1.fit(train1[feature1], train1[target])
prediction1 = lr1.predict(test1[feature1])
mae1 = mean_absolute_error(test1[target], prediction1)

print('Mean Absolute Error for using closing price and volumes:', mae1)

Mean Absolute Error for using closing price and volumes: 15.9851563779
